In [19]:
import json 
import subprocess
import os
WINNERS = []


In [20]:

pieces = ["pawn", "knight", "queen", "bishop", "king", "rook"]
PARAMS = {
    "settings_path": "", 
    "nocapture": False, 
    "max_iter": 50, 
    "recursion_limit": 10000,
    "time_limit": 1000,
    "memory_limit": 20000,
    "absolute_params": {
        "turn_value": -0.0,
        "defended": -0.0,
        "attacked": -0.0,
        "available_moves": 0.00,
        "piece_value": 1.0,
        "mov_value": 0.0,
        "kingside_castle": 0.0,
        "queenside_castle": 0.0,
        "pawn": {
            "white": [[1 for _ in range(8)] for _ in range(8)],
            "black": [[1 for _ in range(8)] for _ in range(8)],
        },
        "knight": {
            "white": [[3 for _ in range(8)] for _ in range(8)],
            "black": [[3 for _ in range(8)] for _ in range(8)],
        },
        "queen": {
            "white": [[9 for _ in range(8)] for _ in range(8)],
            "black": [[9 for _ in range(8)] for _ in range(8)],
        },
        "bishop": {
            "white": [[3.5 for _ in range(8)] for _ in range(8)],
            "black": [[3.5 for _ in range(8)] for _ in range(8)]
        },
        "king": {
            "white": [[100 for _ in range(8)] for _ in range(8)],
            "black": [[100 for _ in range(8)] for _ in range(8)]
        },
        "rook": {
            "white": [[4 for _ in range(8)] for _ in range(8)],
            "black": [[4 for _ in range(8)] for _ in range(8)]
        }
    },
}
WINNERS = [PARAMS["absolute_params"]]*5


In [21]:
import numpy as np
from copy import deepcopy
from random import random

LEARNING_RATE = 3

def random_params(): 
    params = deepcopy(PARAMS)
    p1 = deepcopy(average(WINNERS)) or deepcopy(PARAMS)["absolute_params"]
    p2 = deepcopy(p1)
    for key, val in p1.items():
        if type(val) == float:
            rand = LEARNING_RATE/4 * (np.random.random() - 0.5)
            p1[key] +=  rand
            p2[key] -=  rand
    for piece in pieces: 
        for color in ["black", "white"]: 
            rand = (np.random.random((8,8)) - 0.5) * LEARNING_RATE 
            
            p1[piece][color] += rand
            p2[piece][color] -= rand
            p1[piece][color] = p1[piece][color].tolist()
            p2[piece][color] = p2[piece][color].tolist()
    
    params["white_params"] = p1
    params["black_params"] = p2
    return params


def run(params): 
    handle = subprocess.run(["chess", json.dumps(params)], capture_output=True)
    if handle.stderr: print(handle.stderr, end="")
    return str(handle.stdout, "utf-8").strip()
    
    
def average(winners): 
    out = {}
    for winner in winners: 
        for key, val in winner.items(): 

            if type(val) == float: 
                out[key] = out.get(key, 0.0)
                out[key] += val/len(winners)
            else: 
                for color in ["black", "white"]: 
                    out[key] = out.get(key, {})
                    out[key][color] = out[key].get(color, [[0.0 for _ in range(8)] for _ in range(8)])
                    out[key][color] += np.array(val[color])/len(winners)
                    out[key][color] = out[key][color].tolist()
    return out





In [31]:


def iteration():
    p = random_params()
    
    result = run(p)

    if result == "black": 
        WINNERS.append(p["black_params"])
    elif result == "white": 
        WINNERS.append(p["white_params"])
    else: 
        
        return
    return result == "black"






In [34]:
import threading
import time
import datetime

start = time.time()
hour = 0

def elapsed(): 
    return time.time() - start

def reduce_lr(): 
    global hour, LEARNING_RATE, WINNERS
    if random() < 1/50: 
        WINNERS = [average(WINNERS)]*3

    if hour * 60**2 < elapsed(): 
        LEARNING_RATE /= 2
        hour += 1
        
        
        print(f"LEARNING_RATE: {LEARNING_RATE}")

        WINNERS = [average(WINNERS)]*3

        with open(f"settings {datetime.datetime.now()}.json", "w") as f:
            f.write(json.dumps(average(WINNERS)))

handles = []
count = 1
black = 0
tie = 0
while True: 
    x = iteration()
    if x == None: 
        tie += 1
    else: 
        black += x
    
    reduce_lr()
    print(
        f"count: {count:10}  black rate: {black/count:5} tie: {tie/count:5}", end="\r")
    count += 1
    if elapsed() > 7 * 60**2: 
        break

print(f"count: {count:10}  black rate: {black/count:5} tie: {tie/count:5}")


LEARNING_RATE: 3.0
LEARNING_RATE: 1.5 black rate: 0.6314360770577934 tie: 0.015411558669001752
LEARNING_RATE: 0.75black rate:  0.57 tie: 0.0132472324723247247213181298262
LEARNING_RATE: 0.375lack rate: 0.5515846567534647 tie: 0.013789616418836616
LEARNING_RATE: 0.1875ack rate: 0.48709810048515745 tie: 0.015607269139051064
LEARNING_RATE: 0.09375ck rate: 0.4912377411704378 tie: 0.0172424844561246062
LEARNING_RATE: 0.046875k rate: 0.43648847983711797 tie: 0.014200236843745455
LEARNING_RATE: 0.0234375 rate: 0.3689123980838449 tie: 0.0122518325203394425
count:     114189  black rate: 0.3689059366488891 tie: 0.0122516179316746833


In [33]:
LEARNING_RATE=6

In [84]:
#average(WINNERS)
3


0.046875

In [ ]:
100 / 2.6**7

In [ ]:
print(json.dumps(average(WINNERS)))

In [ ]:
x = [{"asd": [1]}]*2
x[0]["asd"].append(3)
x

In [ ]:
json.dumps(average(WINNERS))

In [ ]:
WINNERS = [{'turn_value': -0.07569701523583391,
 'defended': 0.046810184617385946,
 'attacked': -0.006006635312639091,
 'available_moves': 0.05951355855642724,
 'piece_value': 0.911605434687519,
 'mov_value': 0.011899169890000024,
 'kingside_castle': 0.06045632183163087,
 'queenside_castle': -0.007878104861463606,
 'pawn': {'black': [[0.7698195452978124,
    0.8524001754742926,
    0.885950373261106,
    0.989350993487395,
    1.124686552154142,
    1.0105646431190534,
    1.1526698053054116,
    0.9972171220407777],
   [1.4884245272874832,
    0.8642372235694591,
    0.8752193081360685,
    1.217722983052616,
    1.1462118841427489,
    1.0203690315296976,
    1.076019481313969,
    1.0656541974955176],
   [0.9881759454084813,
    0.9224741799912529,
    0.9219612908908001,
    1.0692025577765927,
    0.7254000266443875,
    1.053817841757842,
    0.9226447413110385,
    0.9648544220519306],
   [0.9452013912398699,
    0.7302250887113214,
    1.1893736370191492,
    0.9725203211021756,
    1.2340398421856331,
    1.0565349001329112,
    1.1200719783057715,
    0.9654759075528752],
   [0.9766032914634983,
    0.9456050032330405,
    0.7762786903058065,
    0.9120841292173524,
    0.8052423833787978,
    0.9738825675584684,
    1.264174919453976,
    0.6619648879629315],
   [0.8365062546618452,
    1.0762379856179196,
    0.7579815474827352,
    0.7164123532140992,
    0.8283520088366786,
    0.9820661035299693,
    1.000898628842393,
    0.970041232188966],
   [0.9657645350239363,
    0.9686887824004629,
    1.2273347247697104,
    0.9533218763139146,
    1.5552290552899444,
    1.0613421836493462,
    0.7458640195677333,
    0.9213082596025413],
   [0.9783245428217022,
    1.128409274029571,
    0.9117050326871972,
    0.9049583942868829,
    1.4456202441326025,
    0.9613266230001964,
    1.1842444908745748,
    1.1280718599475306]],
  'white': [[0.5433019083665538,
    0.7675017378083852,
    0.9752809921683353,
    1.0021163622935418,
    1.0819430752540125,
    0.5303345238883296,
    0.7285976396581225,
    1.0658513672666292],
   [0.8893890879549919,
    1.2901203797783485,
    0.8984256210009772,
    0.9014233785564246,
    1.1426283199607592,
    0.6888400348178465,
    0.8403986825314007,
    1.1551070446240657],
   [1.2992002175750599,
    1.058815309205556,
    1.2620324976526016,
    1.1637709680415325,
    0.9133285626411576,
    0.6026462547513614,
    0.8125719582467433,
    0.7796034510713501],
   [0.9979641653825259,
    0.8850882040973801,
    0.8479158935279864,
    0.8182902091625912,
    1.010061525945901,
    0.9390589120328888,
    1.5602625519956184,
    1.103685389288498],
   [0.9676028728283291,
    1.053923966244108,
    1.5324099400367446,
    0.9990578211795467,
    0.7140481371164848,
    1.0503417057133049,
    0.8094421344291595,
    1.0522748938812159],
   [0.874721113469556,
    1.1863576517045629,
    0.991372993824353,
    0.9092694498325969,
    0.6457322095170327,
    0.9484302952624434,
    1.018580740659088,
    1.2919891862089463],
   [0.8411154987193765,
    1.0263042009463592,
    0.9527483032539866,
    1.0630946814462927,
    0.7320908937151493,
    0.9589133049138732,
    0.7047657271291895,
    0.8023746381205389],
   [1.0281033382117257,
    0.8863200620473928,
    0.9837805351247204,
    0.9365769883455237,
    1.0091074057498584,
    0.7598593352687598,
    0.9701874173492463,
    1.115247167792047]]},
 'knight': {'black': [[3.0977128748913154,
    3.3196950295070766,
    2.919044681599936,
    3.203576590210807,
    2.720791739002768,
    3.0139939723030684,
    2.7464357849366827,
    2.9232998760632007],
   [3.119408855686608,
    2.9902283947006536,
    2.900906979470872,
    2.921019242963496,
    3.2237121479885236,
    2.8251792656513177,
    3.0254715945312576,
    2.795383212959106],
   [2.903913908291213,
    3.0600481751227866,
    2.9198783397055768,
    3.0260276505662267,
    2.8701992423400458,
    3.191237185056882,
    3.0732025724116108,
    3.2959602570117923],
   [3.0725016072876397,
    2.66737784329588,
    2.7611364659621573,
    3.225553703019319,
    3.0776684224080797,
    2.9306882135418664,
    3.2023877871586772,
    3.19444013962084],
   [2.6862519672773693,
    2.9048614584917205,
    3.4246421971974494,
    2.908558347133712,
    2.7016460145385293,
    2.884059701298279,
    3.2110341521625037,
    3.002024315313518],
   [3.1262687905916557,
    3.2220961712916236,
    3.146808089185808,
    2.913397609669656,
    2.953484935385633,
    3.32742873041096,
    2.829998122230676,
    3.0969676026403805],
   [2.9942457690330047,
    3.4448014415504313,
    3.0906852312191138,
    2.788875422579244,
    2.874825068636974,
    3.193857019772658,
    2.901252821163484,
    2.9263479373650347],
   [3.2875401344681126,
    3.036444069332237,
    2.9318537996684384,
    3.2320138955122606,
    2.8436712433074005,
    2.895261764014761,
    2.858452250690155,
    3.1479463917295702]],
  'white': [[2.6436079213910024,
    3.3734846149103315,
    3.0497691848956756,
    2.795708623398274,
    2.779989498098614,
    3.174815371423954,
    3.1733161987111873,
    3.015598480807404],
   [3.096448556415643,
    2.991821990441693,
    2.854458068604301,
    2.880936249727024,
    3.2884985131701123,
    2.880800919287481,
    2.8697952454180524,
    2.977298843557238],
   [2.966070671742914,
    3.164055184647695,
    2.8308029995021173,
    2.9100169161370135,
    3.128390136929134,
    2.9579049338638814,
    3.0912246495616884,
    2.957634359669731],
   [2.8066097419301097,
    3.2950186878020435,
    2.972639898477288,
    2.954825156264905,
    3.1949489441601364,
    3.0700403274637424,
    2.6548342628788375,
    3.1140899393817003],
   [2.888977528872638,
    3.173343743469023,
    3.181138090754391,
    3.072692755451248,
    2.8204247425351783,
    3.080154866881674,
    2.874989685626976,
    2.9670491145591145],
   [3.134919304284218,
    2.956325079910529,
    3.246798601442827,
    3.0296995925788344,
    2.8176472547780107,
    2.86528837999444,
    2.9131430857933247,
    3.071156064954467],
   [2.7909183409566825,
    3.158965419636804,
    3.0061168879746205,
    2.7330388835479766,
    2.6220637752967435,
    3.1084188060496096,
    3.159240868521116,
    2.901793100575439],
   [3.119889267473812,
    3.0131400405591227,
    3.098858406237948,
    2.817733214043578,
    2.894569995391075,
    2.984145919752042,
    3.031662782782063,
    2.6868294981013885]]},
 'queen': {'black': [[8.97001268111041,
    8.931377847731026,
    9.01178694624534,
    9.23619641283269,
    9.195213692566137,
    8.861961453276452,
    8.783032928234224,
    9.088966957063466],
   [8.890529040059329,
    8.912359285307083,
    8.90562854464211,
    8.712399124966135,
    9.361162774345484,
    9.299493695458201,
    9.04063050344236,
    8.90708311939679],
   [8.91466544953635,
    9.019780795961802,
    8.84939865236111,
    9.282647356686837,
    9.130535775428333,
    8.921925015957536,
    9.229408706290766,
    9.046813951591545],
   [8.999796969352255,
    8.985255515566735,
    9.330214811763584,
    8.74052710071817,
    9.133608421043379,
    9.233319487044493,
    9.215759548660595,
    9.00732129357623],
   [9.463259726905799,
    8.833902157366834,
    9.240786687875927,
    9.284046343582805,
    8.594604205591583,
    8.843212203293348,
    8.92150841975139,
    9.073424622665687],
   [9.095168766050707,
    8.516538367531735,
    8.71771683377907,
    8.752454050663319,
    9.058556947730883,
    8.82060246689117,
    9.113500314245805,
    8.877010299045505],
   [9.153730334383305,
    9.19590634242727,
    8.596435547969392,
    9.113646959287268,
    9.051625544849538,
    8.744300121151328,
    9.028579774731412,
    9.17637562452906],
   [8.916114390817462,
    9.08887803493085,
    9.203014057145076,
    8.84946897590021,
    8.919084296249853,
    8.73815302369734,
    9.348565698019094,
    9.244899525169599]],
  'white': [[8.777479846087234,
    8.82161553375098,
    9.089708177615984,
    9.087877676080277,
    8.897291168332925,
    9.029930491218277,
    8.961205293362363,
    8.98642056915703],
   [8.97929758757762,
    8.935050432834704,
    8.71888784853454,
    8.987032360348502,
    9.331344038989197,
    8.740460782783012,
    8.763082864407336,
    8.782023967929804],
   [9.105013191125876,
    9.144828208616014,
    9.063609979116498,
    9.014593874480898,
    8.944039829279435,
    8.881628016273512,
    8.936852729105807,
    8.71010098210599],
   [8.979902069456843,
    9.561862014227696,
    8.883630946203562,
    8.967832309017366,
    8.867428606128538,
    9.113031841424435,
    8.978328388188036,
    8.980145978481756],
   [8.907297009440983,
    8.89179459714044,
    8.77872859141289,
    9.205014036483165,
    8.862772485854112,
    9.091333880996189,
    8.851149098718155,
    8.817454610068577],
   [8.992185115238678,
    8.630897943652977,
    8.76387450703415,
    9.051027868092845,
    9.133050663097299,
    9.06345536342971,
    9.003780426235949,
    9.199418848462495],
   [9.132701620615236,
    8.89610422679414,
    8.886630016211937,
    8.753131600239909,
    8.756377831107475,
    9.23949213649286,
    9.038146846683723,
    8.91259984299117],
   [9.236718244573275,
    8.886375769011435,
    8.73604043634749,
    9.100630899950508,
    9.050943812200906,
    8.795836067884471,
    8.980223286364616,
    9.038994716578802]]},
 'bishop': {'black': [[3.0755972792389197,
    3.075627337720417,
    2.8289347188000664,
    2.6827757738463975,
    3.1120054949089626,
    2.9842300250665987,
    2.596117478786305,
    2.758817069341737],
   [2.9910349301696515,
    3.0245898737754033,
    3.414444536174588,
    2.6615074323275603,
    3.157602006572632,
    3.1387776834470813,
    3.5188631141693523,
    3.0218207079379633],
   [2.850860989298274,
    2.9635413398764414,
    2.971394347718401,
    3.0166123857982483,
    3.038689689442015,
    3.1426963653967244,
    3.137459112987368,
    2.9333437112304526],
   [3.1606983855910524,
    3.0447401594477235,
    3.0868908714433294,
    3.0459953726887212,
    3.05945907172175,
    3.2034544909128564,
    3.0276774150150416,
    3.120652395354132],
   [2.8591808800877994,
    2.7973425545222983,
    2.753036369289704,
    2.800787655609669,
    3.3488315751854216,
    2.9861113355018007,
    2.9602930080813565,
    3.098972876596388],
   [3.090276037210512,
    3.179986322459275,
    2.816273950799243,
    3.085048115667099,
    2.92924021147199,
    2.835405955828285,
    2.466730340026985,
    2.612396294801208],
   [2.979946174439647,
    2.6288828761936496,
    3.0026214169148058,
    3.2271587441763088,
    2.7791619809626,
    2.9222653220671453,
    3.266782486121753,
    3.4198121009631643],
   [3.412674852236752,
    2.9907714384719664,
    2.781598190278125,
    3.1617611018507508,
    2.7679806601055343,
    3.1162170698784295,
    3.1088484847381777,
    3.116766198253499]],
  'white': [[2.7127973015736244,
    3.117414471359191,
    2.974714421856493,
    3.118799598617188,
    2.9057205448508454,
    2.9030094152210424,
    3.0247035043620243,
    3.0472866018373232],
   [2.471642256418242,
    3.166531828610367,
    2.6687678523199123,
    2.942614388300015,
    3.0070790952295505,
    2.8851805514685758,
    2.9395117298713234,
    2.8441023566219017],
   [2.8702006037946455,
    3.060286820118279,
    2.8861619857255834,
    2.9078598868519028,
    2.60239365888104,
    2.884340365404393,
    3.14312548176293,
    2.8161325813181244],
   [2.9160937713121715,
    2.99253062764926,
    3.496182351203352,
    3.249939969852881,
    3.3388962904333623,
    2.792552110445733,
    2.7806462607561246,
    3.3214702162812113],
   [2.8967239329890155,
    3.1187847573756957,
    3.073938541409569,
    2.943013643574248,
    3.1148163806837355,
    3.2471405103769393,
    3.201913890313629,
    3.112423195232328],
   [3.1935221360283754,
    2.8011111374372994,
    2.8539416022854747,
    3.0537480000473916,
    3.356979435094771,
    3.059468516746103,
    3.2763306015747,
    2.9478113365940684],
   [3.010199444451365,
    2.7366095216497306,
    2.7889437485535655,
    3.026342214647424,
    3.3322984681260763,
    2.870617975183187,
    3.0285367123748976,
    2.7937593173281803],
   [3.2329511172634877,
    3.4056404876721915,
    2.874075885413229,
    2.852964848959723,
    3.244695144831668,
    3.1550654287364233,
    2.787754801972232,
    2.825429588475201]]},
 'king': {'black': [[100.0226903993703,
    100.21642129406226,
    99.84854492331027,
    100.25670004876311,
    100.07900123142736,
    99.93508660246847,
    100.14714176303397,
    100.13582959559125],
   [100.0176192754921,
    100.24870926782752,
    100.22479132968448,
    99.86647093521819,
    100.73084147828027,
    100.18372693237075,
    100.1338418033323,
    100.09439735583096],
   [100.1065870534225,
    100.14453212399481,
    99.90418104844846,
    99.88707238286423,
    99.79831793074698,
    99.95442897843972,
    100.06877876731801,
    99.78888181777945],
   [100.00441729907938,
    99.93027565131983,
    99.81204960278718,
    99.97667701987027,
    100.07861331980796,
    99.95163929593598,
    100.02757911810252,
    100.07851466990222],
   [99.80397690467521,
    99.66556681381142,
    99.85700677528426,
    100.05006190774763,
    100.06526693460762,
    100.21069529910888,
    100.20684886562783,
    99.95187267641984],
   [99.98155981717437,
    100.11709116715058,
    99.97544532793026,
    99.95314296432842,
    100.10116990108412,
    99.97061910299563,
    99.83621597735267,
    99.7011000186074],
   [99.81952111022414,
    100.34680298962759,
    99.97959682625812,
    99.90740408660682,
    99.79285246264888,
    100.00203078549296,
    100.38850328247082,
    99.88811292170388],
   [100.12605166563384,
    100.22485326808977,
    100.20272751438831,
    100.08287988826574,
    99.62410970972742,
    99.88083904803725,
    99.77207056901443,
    100.17100024715653]],
  'white': [[99.7842736249761,
    100.19552730198474,
    100.23621956920869,
    100.15172813562027,
    99.93487070879422,
    100.04647352793374,
    100.29521539146478,
    100.3849759142224],
   [100.0372884617271,
    100.12281609059424,
    100.0345360033051,
    99.92040307767067,
    100.12040937382187,
    100.07035530198772,
    99.79118915353004,
    100.04841437699837],
   [99.99842934668273,
    99.92201757297771,
    99.83478502076899,
    99.89570201813079,
    99.98263486443285,
    99.79805354589053,
    99.96905578794788,
    99.6769415239412],
   [100.22045633385571,
    100.15186134314509,
    99.87289016049613,
    100.14101616862523,
    99.95847432475692,
    100.0919615754528,
    100.03404181935385,
    100.13179983687286],
   [99.49524614126635,
    99.90588843871478,
    100.14215566402774,
    100.16772056861846,
    99.98919444154451,
    99.98781483267577,
    99.86711438818858,
    99.79329911321805],
   [100.24057325748542,
    99.98582664777409,
    99.89378303826318,
    100.03618239829483,
    99.97950149576775,
    100.15905606553736,
    99.93162470186573,
    100.04604104874716],
   [99.89730602655617,
    100.02895748407887,
    99.65088260112589,
    99.74446128377656,
    99.76472549264855,
    99.82105613720731,
    99.80883515014295,
    100.265650660926],
   [100.0452885506297,
    99.85419254167421,
    99.99533669561659,
    100.1554737282652,
    100.00771176293706,
    100.39207991436083,
    100.05139044730608,
    99.90567782743285]]},
 'rook': {'black': [[4.065034192556363,
    4.033661768169822,
    4.014603611847306,
    4.201989145020364,
    3.8439497636439874,
    3.8581321084186384,
    4.076084547195674,
    3.5704386190738995],
   [3.886334278706779,
    3.658800215389784,
    3.532213483328312,
    4.321903368292668,
    4.161846450598257,
    3.9563202524524193,
    4.055750294870151,
    3.987193522411922],
   [4.188052093044711,
    4.3129802683586504,
    4.0713941333182895,
    3.8379275965180204,
    3.8280881547591035,
    3.774110554278353,
    3.7938419581785756,
    4.02112070488732],
   [3.998072997687834,
    3.8797865311453084,
    3.9153092555008393,
    3.870029870205165,
    4.006998587157087,
    4.083464041356784,
    3.7085562174676774,
    3.926267984653815],
   [4.175486224940896,
    3.917178950976,
    3.668821821062816,
    4.333100416351902,
    4.159063596669919,
    4.31808278615558,
    3.706608959879005,
    3.8913309975072363],
   [3.8632091735370415,
    3.9533759469842926,
    4.167697559312532,
    4.2035690774647465,
    4.123001466804213,
    3.905748898170517,
    3.9829490782000843,
    3.8243974807235652],
   [4.007466873190862,
    3.8844495426383596,
    3.9111889231726695,
    4.021657592324268,
    3.9364493552374635,
    4.160527699819028,
    4.095400564459501,
    3.9579401564518775],
   [4.060165121384941,
    4.028004938810268,
    3.96485519765459,
    4.209157538475296,
    3.9030370073922565,
    4.02420465688945,
    4.13426255125879,
    4.080426063741675]],
  'white': [[3.901871552542451,
    3.844618134701836,
    4.236574411738991,
    3.9645800970260385,
    4.133442796296916,
    4.0729417958093155,
    4.252570157043821,
    3.742521847265909],
   [4.0014859530120646,
    4.064739652610799,
    4.464767473189358,
    4.082697383557691,
    3.9724887513142555,
    3.686076040527715,
    3.7935380524937443,
    4.014297842594801],
   [3.7877522508633197,
    4.062749543331331,
    4.108405291282168,
    4.0755219277878165,
    3.884841438673626,
    4.124727250199452,
    3.9942590840747756,
    3.8809815653155133],
   [3.7438373482862266,
    4.387884166665905,
    4.11805873729208,
    3.955412831009131,
    3.9320589151127576,
    3.895881464361889,
    3.834391472503052,
    3.7247552226102867],
   [4.085319288104365,
    4.07790591139316,
    4.096383047752838,
    4.205489050053652,
    4.1226767829220545,
    3.9524166818741246,
    3.8150985315216004,
    4.2476453682629955],
   [4.127413796061171,
    4.143909471487992,
    4.104869147275554,
    3.9426563996170056,
    3.7449899491869973,
    3.987120943020637,
    3.810912491376929,
    4.290292693245236],
   [3.6953450970354824,
    3.930047516657562,
    4.284968895253137,
    4.176347743357431,
    3.974460370952621,
    4.038556682547983,
    3.986176435459408,
    4.15905694665821],
   [3.8870731100507396,
    3.846793371061618,
    3.9552984698652573,
    4.2597614367621075,
    4.076934175205623,
    3.8322531220047935,
    3.9876761409548105,
    4.318891994536892]]}}]

In [ ]:
json.dumps({'turn_value': -0.07569701523583391,
 'defended': 0.046810184617385946,
 'attacked': -0.006006635312639091,
 'available_moves': 0.05951355855642724,
 'piece_value': 0.911605434687519,
 'mov_value': 0.011899169890000024,
 'kingside_castle': 0.06045632183163087,
 'queenside_castle': -0.007878104861463606,
 'pawn': {'black': [[0.7698195452978124,
    0.8524001754742926,
    0.885950373261106,
    0.989350993487395,
    1.124686552154142,
    1.0105646431190534,
    1.1526698053054116,
    0.9972171220407777],
   [1.4884245272874832,
    0.8642372235694591,
    0.8752193081360685,
    1.217722983052616,
    1.1462118841427489,
    1.0203690315296976,
    1.076019481313969,
    1.0656541974955176],
   [0.9881759454084813,
    0.9224741799912529,
    0.9219612908908001,
    1.0692025577765927,
    0.7254000266443875,
    1.053817841757842,
    0.9226447413110385,
    0.9648544220519306],
   [0.9452013912398699,
    0.7302250887113214,
    1.1893736370191492,
    0.9725203211021756,
    1.2340398421856331,
    1.0565349001329112,
    1.1200719783057715,
    0.9654759075528752],
   [0.9766032914634983,
    0.9456050032330405,
    0.7762786903058065,
    0.9120841292173524,
    0.8052423833787978,
    0.9738825675584684,
    1.264174919453976,
    0.6619648879629315],
   [0.8365062546618452,
    1.0762379856179196,
    0.7579815474827352,
    0.7164123532140992,
    0.8283520088366786,
    0.9820661035299693,
    1.000898628842393,
    0.970041232188966],
   [0.9657645350239363,
    0.9686887824004629,
    1.2273347247697104,
    0.9533218763139146,
    1.5552290552899444,
    1.0613421836493462,
    0.7458640195677333,
    0.9213082596025413],
   [0.9783245428217022,
    1.128409274029571,
    0.9117050326871972,
    0.9049583942868829,
    1.4456202441326025,
    0.9613266230001964,
    1.1842444908745748,
    1.1280718599475306]],
  'white': [[0.5433019083665538,
    0.7675017378083852,
    0.9752809921683353,
    1.0021163622935418,
    1.0819430752540125,
    0.5303345238883296,
    0.7285976396581225,
    1.0658513672666292],
   [0.8893890879549919,
    1.2901203797783485,
    0.8984256210009772,
    0.9014233785564246,
    1.1426283199607592,
    0.6888400348178465,
    0.8403986825314007,
    1.1551070446240657],
   [1.2992002175750599,
    1.058815309205556,
    1.2620324976526016,
    1.1637709680415325,
    0.9133285626411576,
    0.6026462547513614,
    0.8125719582467433,
    0.7796034510713501],
   [0.9979641653825259,
    0.8850882040973801,
    0.8479158935279864,
    0.8182902091625912,
    1.010061525945901,
    0.9390589120328888,
    1.5602625519956184,
    1.103685389288498],
   [0.9676028728283291,
    1.053923966244108,
    1.5324099400367446,
    0.9990578211795467,
    0.7140481371164848,
    1.0503417057133049,
    0.8094421344291595,
    1.0522748938812159],
   [0.874721113469556,
    1.1863576517045629,
    0.991372993824353,
    0.9092694498325969,
    0.6457322095170327,
    0.9484302952624434,
    1.018580740659088,
    1.2919891862089463],
   [0.8411154987193765,
    1.0263042009463592,
    0.9527483032539866,
    1.0630946814462927,
    0.7320908937151493,
    0.9589133049138732,
    0.7047657271291895,
    0.8023746381205389],
   [1.0281033382117257,
    0.8863200620473928,
    0.9837805351247204,
    0.9365769883455237,
    1.0091074057498584,
    0.7598593352687598,
    0.9701874173492463,
    1.115247167792047]]},
 'knight': {'black': [[3.0977128748913154,
    3.3196950295070766,
    2.919044681599936,
    3.203576590210807,
    2.720791739002768,
    3.0139939723030684,
    2.7464357849366827,
    2.9232998760632007],
   [3.119408855686608,
    2.9902283947006536,
    2.900906979470872,
    2.921019242963496,
    3.2237121479885236,
    2.8251792656513177,
    3.0254715945312576,
    2.795383212959106],
   [2.903913908291213,
    3.0600481751227866,
    2.9198783397055768,
    3.0260276505662267,
    2.8701992423400458,
    3.191237185056882,
    3.0732025724116108,
    3.2959602570117923],
   [3.0725016072876397,
    2.66737784329588,
    2.7611364659621573,
    3.225553703019319,
    3.0776684224080797,
    2.9306882135418664,
    3.2023877871586772,
    3.19444013962084],
   [2.6862519672773693,
    2.9048614584917205,
    3.4246421971974494,
    2.908558347133712,
    2.7016460145385293,
    2.884059701298279,
    3.2110341521625037,
    3.002024315313518],
   [3.1262687905916557,
    3.2220961712916236,
    3.146808089185808,
    2.913397609669656,
    2.953484935385633,
    3.32742873041096,
    2.829998122230676,
    3.0969676026403805],
   [2.9942457690330047,
    3.4448014415504313,
    3.0906852312191138,
    2.788875422579244,
    2.874825068636974,
    3.193857019772658,
    2.901252821163484,
    2.9263479373650347],
   [3.2875401344681126,
    3.036444069332237,
    2.9318537996684384,
    3.2320138955122606,
    2.8436712433074005,
    2.895261764014761,
    2.858452250690155,
    3.1479463917295702]],
  'white': [[2.6436079213910024,
    3.3734846149103315,
    3.0497691848956756,
    2.795708623398274,
    2.779989498098614,
    3.174815371423954,
    3.1733161987111873,
    3.015598480807404],
   [3.096448556415643,
    2.991821990441693,
    2.854458068604301,
    2.880936249727024,
    3.2884985131701123,
    2.880800919287481,
    2.8697952454180524,
    2.977298843557238],
   [2.966070671742914,
    3.164055184647695,
    2.8308029995021173,
    2.9100169161370135,
    3.128390136929134,
    2.9579049338638814,
    3.0912246495616884,
    2.957634359669731],
   [2.8066097419301097,
    3.2950186878020435,
    2.972639898477288,
    2.954825156264905,
    3.1949489441601364,
    3.0700403274637424,
    2.6548342628788375,
    3.1140899393817003],
   [2.888977528872638,
    3.173343743469023,
    3.181138090754391,
    3.072692755451248,
    2.8204247425351783,
    3.080154866881674,
    2.874989685626976,
    2.9670491145591145],
   [3.134919304284218,
    2.956325079910529,
    3.246798601442827,
    3.0296995925788344,
    2.8176472547780107,
    2.86528837999444,
    2.9131430857933247,
    3.071156064954467],
   [2.7909183409566825,
    3.158965419636804,
    3.0061168879746205,
    2.7330388835479766,
    2.6220637752967435,
    3.1084188060496096,
    3.159240868521116,
    2.901793100575439],
   [3.119889267473812,
    3.0131400405591227,
    3.098858406237948,
    2.817733214043578,
    2.894569995391075,
    2.984145919752042,
    3.031662782782063,
    2.6868294981013885]]},
 'queen': {'black': [[8.97001268111041,
    8.931377847731026,
    9.01178694624534,
    9.23619641283269,
    9.195213692566137,
    8.861961453276452,
    8.783032928234224,
    9.088966957063466],
   [8.890529040059329,
    8.912359285307083,
    8.90562854464211,
    8.712399124966135,
    9.361162774345484,
    9.299493695458201,
    9.04063050344236,
    8.90708311939679],
   [8.91466544953635,
    9.019780795961802,
    8.84939865236111,
    9.282647356686837,
    9.130535775428333,
    8.921925015957536,
    9.229408706290766,
    9.046813951591545],
   [8.999796969352255,
    8.985255515566735,
    9.330214811763584,
    8.74052710071817,
    9.133608421043379,
    9.233319487044493,
    9.215759548660595,
    9.00732129357623],
   [9.463259726905799,
    8.833902157366834,
    9.240786687875927,
    9.284046343582805,
    8.594604205591583,
    8.843212203293348,
    8.92150841975139,
    9.073424622665687],
   [9.095168766050707,
    8.516538367531735,
    8.71771683377907,
    8.752454050663319,
    9.058556947730883,
    8.82060246689117,
    9.113500314245805,
    8.877010299045505],
   [9.153730334383305,
    9.19590634242727,
    8.596435547969392,
    9.113646959287268,
    9.051625544849538,
    8.744300121151328,
    9.028579774731412,
    9.17637562452906],
   [8.916114390817462,
    9.08887803493085,
    9.203014057145076,
    8.84946897590021,
    8.919084296249853,
    8.73815302369734,
    9.348565698019094,
    9.244899525169599]],
  'white': [[8.777479846087234,
    8.82161553375098,
    9.089708177615984,
    9.087877676080277,
    8.897291168332925,
    9.029930491218277,
    8.961205293362363,
    8.98642056915703],
   [8.97929758757762,
    8.935050432834704,
    8.71888784853454,
    8.987032360348502,
    9.331344038989197,
    8.740460782783012,
    8.763082864407336,
    8.782023967929804],
   [9.105013191125876,
    9.144828208616014,
    9.063609979116498,
    9.014593874480898,
    8.944039829279435,
    8.881628016273512,
    8.936852729105807,
    8.71010098210599],
   [8.979902069456843,
    9.561862014227696,
    8.883630946203562,
    8.967832309017366,
    8.867428606128538,
    9.113031841424435,
    8.978328388188036,
    8.980145978481756],
   [8.907297009440983,
    8.89179459714044,
    8.77872859141289,
    9.205014036483165,
    8.862772485854112,
    9.091333880996189,
    8.851149098718155,
    8.817454610068577],
   [8.992185115238678,
    8.630897943652977,
    8.76387450703415,
    9.051027868092845,
    9.133050663097299,
    9.06345536342971,
    9.003780426235949,
    9.199418848462495],
   [9.132701620615236,
    8.89610422679414,
    8.886630016211937,
    8.753131600239909,
    8.756377831107475,
    9.23949213649286,
    9.038146846683723,
    8.91259984299117],
   [9.236718244573275,
    8.886375769011435,
    8.73604043634749,
    9.100630899950508,
    9.050943812200906,
    8.795836067884471,
    8.980223286364616,
    9.038994716578802]]},
 'bishop': {'black': [[3.0755972792389197,
    3.075627337720417,
    2.8289347188000664,
    2.6827757738463975,
    3.1120054949089626,
    2.9842300250665987,
    2.596117478786305,
    2.758817069341737],
   [2.9910349301696515,
    3.0245898737754033,
    3.414444536174588,
    2.6615074323275603,
    3.157602006572632,
    3.1387776834470813,
    3.5188631141693523,
    3.0218207079379633],
   [2.850860989298274,
    2.9635413398764414,
    2.971394347718401,
    3.0166123857982483,
    3.038689689442015,
    3.1426963653967244,
    3.137459112987368,
    2.9333437112304526],
   [3.1606983855910524,
    3.0447401594477235,
    3.0868908714433294,
    3.0459953726887212,
    3.05945907172175,
    3.2034544909128564,
    3.0276774150150416,
    3.120652395354132],
   [2.8591808800877994,
    2.7973425545222983,
    2.753036369289704,
    2.800787655609669,
    3.3488315751854216,
    2.9861113355018007,
    2.9602930080813565,
    3.098972876596388],
   [3.090276037210512,
    3.179986322459275,
    2.816273950799243,
    3.085048115667099,
    2.92924021147199,
    2.835405955828285,
    2.466730340026985,
    2.612396294801208],
   [2.979946174439647,
    2.6288828761936496,
    3.0026214169148058,
    3.2271587441763088,
    2.7791619809626,
    2.9222653220671453,
    3.266782486121753,
    3.4198121009631643],
   [3.412674852236752,
    2.9907714384719664,
    2.781598190278125,
    3.1617611018507508,
    2.7679806601055343,
    3.1162170698784295,
    3.1088484847381777,
    3.116766198253499]],
  'white': [[2.7127973015736244,
    3.117414471359191,
    2.974714421856493,
    3.118799598617188,
    2.9057205448508454,
    2.9030094152210424,
    3.0247035043620243,
    3.0472866018373232],
   [2.471642256418242,
    3.166531828610367,
    2.6687678523199123,
    2.942614388300015,
    3.0070790952295505,
    2.8851805514685758,
    2.9395117298713234,
    2.8441023566219017],
   [2.8702006037946455,
    3.060286820118279,
    2.8861619857255834,
    2.9078598868519028,
    2.60239365888104,
    2.884340365404393,
    3.14312548176293,
    2.8161325813181244],
   [2.9160937713121715,
    2.99253062764926,
    3.496182351203352,
    3.249939969852881,
    3.3388962904333623,
    2.792552110445733,
    2.7806462607561246,
    3.3214702162812113],
   [2.8967239329890155,
    3.1187847573756957,
    3.073938541409569,
    2.943013643574248,
    3.1148163806837355,
    3.2471405103769393,
    3.201913890313629,
    3.112423195232328],
   [3.1935221360283754,
    2.8011111374372994,
    2.8539416022854747,
    3.0537480000473916,
    3.356979435094771,
    3.059468516746103,
    3.2763306015747,
    2.9478113365940684],
   [3.010199444451365,
    2.7366095216497306,
    2.7889437485535655,
    3.026342214647424,
    3.3322984681260763,
    2.870617975183187,
    3.0285367123748976,
    2.7937593173281803],
   [3.2329511172634877,
    3.4056404876721915,
    2.874075885413229,
    2.852964848959723,
    3.244695144831668,
    3.1550654287364233,
    2.787754801972232,
    2.825429588475201]]},
 'king': {'black': [[100.0226903993703,
    100.21642129406226,
    99.84854492331027,
    100.25670004876311,
    100.07900123142736,
    99.93508660246847,
    100.14714176303397,
    100.13582959559125],
   [100.0176192754921,
    100.24870926782752,
    100.22479132968448,
    99.86647093521819,
    100.73084147828027,
    100.18372693237075,
    100.1338418033323,
    100.09439735583096],
   [100.1065870534225,
    100.14453212399481,
    99.90418104844846,
    99.88707238286423,
    99.79831793074698,
    99.95442897843972,
    100.06877876731801,
    99.78888181777945],
   [100.00441729907938,
    99.93027565131983,
    99.81204960278718,
    99.97667701987027,
    100.07861331980796,
    99.95163929593598,
    100.02757911810252,
    100.07851466990222],
   [99.80397690467521,
    99.66556681381142,
    99.85700677528426,
    100.05006190774763,
    100.06526693460762,
    100.21069529910888,
    100.20684886562783,
    99.95187267641984],
   [99.98155981717437,
    100.11709116715058,
    99.97544532793026,
    99.95314296432842,
    100.10116990108412,
    99.97061910299563,
    99.83621597735267,
    99.7011000186074],
   [99.81952111022414,
    100.34680298962759,
    99.97959682625812,
    99.90740408660682,
    99.79285246264888,
    100.00203078549296,
    100.38850328247082,
    99.88811292170388],
   [100.12605166563384,
    100.22485326808977,
    100.20272751438831,
    100.08287988826574,
    99.62410970972742,
    99.88083904803725,
    99.77207056901443,
    100.17100024715653]],
  'white': [[99.7842736249761,
    100.19552730198474,
    100.23621956920869,
    100.15172813562027,
    99.93487070879422,
    100.04647352793374,
    100.29521539146478,
    100.3849759142224],
   [100.0372884617271,
    100.12281609059424,
    100.0345360033051,
    99.92040307767067,
    100.12040937382187,
    100.07035530198772,
    99.79118915353004,
    100.04841437699837],
   [99.99842934668273,
    99.92201757297771,
    99.83478502076899,
    99.89570201813079,
    99.98263486443285,
    99.79805354589053,
    99.96905578794788,
    99.6769415239412],
   [100.22045633385571,
    100.15186134314509,
    99.87289016049613,
    100.14101616862523,
    99.95847432475692,
    100.0919615754528,
    100.03404181935385,
    100.13179983687286],
   [99.49524614126635,
    99.90588843871478,
    100.14215566402774,
    100.16772056861846,
    99.98919444154451,
    99.98781483267577,
    99.86711438818858,
    99.79329911321805],
   [100.24057325748542,
    99.98582664777409,
    99.89378303826318,
    100.03618239829483,
    99.97950149576775,
    100.15905606553736,
    99.93162470186573,
    100.04604104874716],
   [99.89730602655617,
    100.02895748407887,
    99.65088260112589,
    99.74446128377656,
    99.76472549264855,
    99.82105613720731,
    99.80883515014295,
    100.265650660926],
   [100.0452885506297,
    99.85419254167421,
    99.99533669561659,
    100.1554737282652,
    100.00771176293706,
    100.39207991436083,
    100.05139044730608,
    99.90567782743285]]},
 'rook': {'black': [[4.065034192556363,
    4.033661768169822,
    4.014603611847306,
    4.201989145020364,
    3.8439497636439874,
    3.8581321084186384,
    4.076084547195674,
    3.5704386190738995],
   [3.886334278706779,
    3.658800215389784,
    3.532213483328312,
    4.321903368292668,
    4.161846450598257,
    3.9563202524524193,
    4.055750294870151,
    3.987193522411922],
   [4.188052093044711,
    4.3129802683586504,
    4.0713941333182895,
    3.8379275965180204,
    3.8280881547591035,
    3.774110554278353,
    3.7938419581785756,
    4.02112070488732],
   [3.998072997687834,
    3.8797865311453084,
    3.9153092555008393,
    3.870029870205165,
    4.006998587157087,
    4.083464041356784,
    3.7085562174676774,
    3.926267984653815],
   [4.175486224940896,
    3.917178950976,
    3.668821821062816,
    4.333100416351902,
    4.159063596669919,
    4.31808278615558,
    3.706608959879005,
    3.8913309975072363],
   [3.8632091735370415,
    3.9533759469842926,
    4.167697559312532,
    4.2035690774647465,
    4.123001466804213,
    3.905748898170517,
    3.9829490782000843,
    3.8243974807235652],
   [4.007466873190862,
    3.8844495426383596,
    3.9111889231726695,
    4.021657592324268,
    3.9364493552374635,
    4.160527699819028,
    4.095400564459501,
    3.9579401564518775],
   [4.060165121384941,
    4.028004938810268,
    3.96485519765459,
    4.209157538475296,
    3.9030370073922565,
    4.02420465688945,
    4.13426255125879,
    4.080426063741675]],
  'white': [[3.901871552542451,
    3.844618134701836,
    4.236574411738991,
    3.9645800970260385,
    4.133442796296916,
    4.0729417958093155,
    4.252570157043821,
    3.742521847265909],
   [4.0014859530120646,
    4.064739652610799,
    4.464767473189358,
    4.082697383557691,
    3.9724887513142555,
    3.686076040527715,
    3.7935380524937443,
    4.014297842594801],
   [3.7877522508633197,
    4.062749543331331,
    4.108405291282168,
    4.0755219277878165,
    3.884841438673626,
    4.124727250199452,
    3.9942590840747756,
    3.8809815653155133],
   [3.7438373482862266,
    4.387884166665905,
    4.11805873729208,
    3.955412831009131,
    3.9320589151127576,
    3.895881464361889,
    3.834391472503052,
    3.7247552226102867],
   [4.085319288104365,
    4.07790591139316,
    4.096383047752838,
    4.205489050053652,
    4.1226767829220545,
    3.9524166818741246,
    3.8150985315216004,
    4.2476453682629955],
   [4.127413796061171,
    4.143909471487992,
    4.104869147275554,
    3.9426563996170056,
    3.7449899491869973,
    3.987120943020637,
    3.810912491376929,
    4.290292693245236],
   [3.6953450970354824,
    3.930047516657562,
    4.284968895253137,
    4.176347743357431,
    3.974460370952621,
    4.038556682547983,
    3.986176435459408,
    4.15905694665821],
   [3.8870731100507396,
    3.846793371061618,
    3.9552984698652573,
    4.2597614367621075,
    4.076934175205623,
    3.8322531220047935,
    3.9876761409548105,
    4.318891994536892]]}})